# Zobecněný pseudopalindromický uzávěr

## Funkce pro vytvoření zobecněného pseudopalindromického uzávěru

In [1]:
import time
import itertools
import re

In [2]:
verbose = True
verboseprint = print if verbose else lambda *a, **k: None

In [3]:
def isPal(seq):
    "kontroluje jestli řetězec je palindrom"
    l = len(seq)
    if l == 1:
        return(True)
    for x in range(0, l//2):
        if seq[x] != seq[l-1-x]:
            return(False)
    return(True)

def isEpal(seq):
    "kontroluje jestli řetězec je pseudopalindrom"
    l = len(seq)
    if l%2 == 1:
        return(False)
    for x in range(0, l//2):
        if seq[x] == seq[l-1-x]:
            return(False)
    return(True)

In [4]:
def makePalClosure (seq):
    "udělá z řetězce palindromický uzávěr"
    if isPal(seq) == True:
        return(seq)
    i = 1
    while isPal(seq[i:]) != True:
        i = i+1
    #print("    {0} nejdelší palindromický uzávěr : {1}".format(seq,seq[i:]))
    #print("    délka nejdelší palindromický uzávěr : {0}".format(len(seq[i:])))
    closure = seq + seq[i-1::-1]
    return(closure)

def makeEpalClosure (seq):
    "udělá z řetězce pseudopalindromický uzávěr"
    if isEpal(seq) == True:
        return(seq)
    i = 1
    while isEpal(seq[i:]) != True:
        i = i+1
    #print("    {0} nejdelší pseudopalindromický uzávěr : {1}".format(seq,seq[i:]))
    closure = seq
    pref = seq[i-1::-1]
    for letter in pref:
        if letter == "0":
            closure = closure + "1"
        if letter == "1":
            closure = closure + "0"
    return(closure)

In [5]:
def makeWord(delta, theta, steps, seed = ""):
    "vytvoří slovo pomocí řídící posloupnosti a posloupnosti uzávěrů"
    w = seed
    for step in range(0,steps):
        w = w + delta[step]
        if theta[step] == "R":
            w = makePalClosure(w)
        if theta[step] == "E":
            w = makeEpalClosure(w)
        #print("w{0} = {1}".format(step+1,w))
    return(w)

In [6]:
def makeS(word):
    "udělá operaci S na slovo"
    Sword = ""
    for i in range(0,len(word)-1):
        Sword += str((int(word[i]) + int(word[i+1])) %2)
    return Sword

In [20]:
def isZps(word, closure = "ER", max_no_matters_closure_type = 0):
    '''kontroluje, jestli možné, aby slovo bylo získané zobec. pal. uzávěrem,
    pokud ano, vrací normalizovanou bidirektivní posloupnost''' 
    maximum = max_no_matters_closure_type
    l=1
    prefixes = []
    while l <= len(word):
        if ((l<=maximum or closure != "E") and isPal(word[:l])) or \
        ((l<=maximum or closure != "R") and isEpal(word[:l])):
            prefixes.append(word[:l])
        l=l+1
    #print(prefixes)
    
    if not prefixes:
        #print("No prefixes of type " + str(closure) + " were found")
        return([False])
    if (len(prefixes[0]) > 2) or (len(prefixes[-1]) < len(word)//2) :
        return([False])        
        
    iszps = True
    i=0
    if closure != "E":
        newtheta = "R"
        newdelta = prefixes[0]
    else:
        newtheta = "E"
        newdelta = prefixes[0][0]
        
    while(i+1 < len(prefixes) and iszps == True):
        newletter = prefixes[i+1][len(prefixes[i])]
        condition_noE = (len(prefixes[i+1]) <= maximum) or (closure != "E")
        condition_noR = (len(prefixes[i+1]) <= maximum) or (closure != "R")
        if (condition_noE):
            palclo = makePalClosure(prefixes[i]+ newletter)
        if (condition_noR):
            epalclo = makeEpalClosure(prefixes[i]+ newletter)

        if(condition_noE and palclo == prefixes[i+1]):
            newtheta = newtheta + "R"
            newdelta = newdelta + newletter
        elif(condition_noR and epalclo == prefixes[i+1]):
            newtheta = newtheta + "E"
            newdelta = newdelta + newletter
        else:
            iszps = False
        i = i+1
    return([iszps, newdelta, newtheta])

In [8]:
def isZP(word):
    '''kontroluje, jestli možné, aby slovo bylo získané R-pal. uzávěrem,
    pokud ano, vrací normalizovanou bidirektivní posloupnost''' 
    l=1
    prefixes = []
    while l <= len(word):
        if isPal(word[:l]):
            prefixes.append(word[:l])
        l=l+1
    #print(prefixes)
    if not prefixes:
        #print("No prefixes of type " + str(closure) + " were found")
        return([False])
    if (len(prefixes[-1]) < len(word)//2) :
        return([False])
        
    iszps = True
    i=0
    newtheta= "R"
    newdelta= prefixes[0]
    while(i+1 < len(prefixes) and iszps == True):
        newletter = prefixes[i+1][len(prefixes[i])]
        palclo = makePalClosure(prefixes[i]+ newletter)

        if(palclo == prefixes[i+1]):
            newtheta = newtheta + "R"
            newdelta = newdelta + newletter
        else:
            iszps = False
        i = i+1
    return([iszps, newdelta, newtheta])

In [9]:
def isZE(word):
    '''kontroluje, jestli možné, aby slovo bylo získané E-uzávěrem,
    pokud ano, vrací normalizovanou bidirektivní posloupnost''' 
    l=1
    prefixes = []
    while l <= len(word):
        if isEpal(word[:l]):
            prefixes.append(word[:l])
        l=l+1
    #print(prefixes)
    if not prefixes:
        #print("No prefixes of type " + str(closure) + " were found")
        return([False])
    if (len(prefixes[-1]) < len(word)//2) :
        return([False])
    if (len(prefixes[0]) != 2):
        return([False])
        
    iszps = True
    i=0
    newtheta = "E"
    newdelta = prefixes[0][0]
    while(i+1 < len(prefixes) and iszps == True):
        newletter = prefixes[i+1][len(prefixes[i])]
        epalclo = makeEpalClosure(prefixes[i]+ newletter)

        if(epalclo == prefixes[i+1]):
            newtheta = newtheta + "E"
            newdelta = newdelta + newletter
        else:
            iszps = False
        i = i+1
    return([iszps, newdelta, newtheta])

In [10]:
def timing(f):
    def wrap(*args):
        time1 = time.time()
        ret = f(*args)
        time2 = time.time()
        print('%s function took %0.3f ms' % (f.__name__, (time2-time1)*1000.0))
        return(ret)
    return wrap

In [11]:
bad_prefixes = ["(0R)*0E", "(1R)*1E", "(0R)+1E1E", "(1R)+0E0E"]
bad_factors = ["1R0E1E", "1R1E0E", "0R0E1E", "0R1E0E", "1E0R1R", "1E1R0R", "0E0R1R", "0E1R0R"]
def makeBiseq(delta, theta):
    """Makes one sequence from tje bi-sequence delta andm theta"""
    if len(delta) != len(theta):
            print("délky delta a theta nejsou stejné")
            return 
    s = ""
    for i in range(len(delta)):
        s = s + delta[i] + theta[i]
    return s

def isNormalized(delta, theta):
    biseq = makeBiseq(delta, theta)
    if biseq.startswith("0R1R") or biseq.startswith("1R0R"):
        print("je tam RR aa*")
        return False
    elif (re.match(bad_prefixes[0], biseq) != None) or (re.match(bad_prefixes[1], biseq) != None):
        print("je tam R^iE a^i")
        return False
    elif (re.match(bad_prefixes[2], biseq) != None) or (re.match(bad_prefixes[3], biseq) != None):
        print("je tam R^iEE a^ia*a*")
        return False
    elif any(x in biseq for x in bad_factors):
        print("ve slove je tt*t* abb*")
        return False
    else:
        return True

In [12]:
@timing
def testGPW_S_on_GPW(deltas, thetas, steps, seed = "", normalized = False, 
                      closure = "RE", max_no_matters_closure_type = 0):
    """Funkce, která všechny delty a thety otestuje, uděla prefixy, operaci S a pak vyzkouší,
    jestli získané slovo může být z zobec. pal. uz."""
    
    printed = False
    for delta in deltas:
        for theta in thetas:
            if(normalized == False or isNormalized(delta, theta)):
                word = makeWord(delta, theta, steps, seed)
                Sword = makeS(word)
                result = isZps(Sword, closure, max_no_matters_closure_type)
                if result[0]==True:
                    #print("Slovo u:" + word)
                    #print("S(u):" + Sword)
                    print("delta = {0}, theta = {1} : {2} ".format(delta, theta, result))
        print("")

In [13]:
# Počet kroků
steps = 2
# Všechny kombinace pro delty a thety
thetas = [''.join(i) for i in itertools.product('ER', repeat=steps)]
deltas = [''.join(i) for i in itertools.product('01', repeat=steps)]

In [14]:
word = makeWord("0111001010", "REERRRRRR", 8)
print(word)
result = isZps(word,"R", 1)
print(result)

01100110011001101100110011001100110110011001100110
[False, '0', 'R']


In [23]:
# Funkce, která všechny delty a thety otestuje, uděla prefixy, operaci S a pak vyzkouší,
# jestli získané slovo může být z zobec. pal. uz.
for delta in deltas:
        for theta in thetas:
            word = makeWord(delta, theta, steps)
            result = isZps(word)
            result2 = isZE(word)
            #print(delta, theta, result, result2)
            if result[0]== True:
                print(delta, theta, result, result2)

00 EE [True, '0101', 'RERE'] [True, '00', 'EE']
00 ER [True, '010', 'RER'] [True, '0', 'E']
00 RE [True, '001', 'RRE'] [False]
00 RR [True, '00', 'RR'] [False]
01 EE [True, '0110', 'RERE'] [True, '01', 'EE']
01 ER [True, '011', 'RER'] [True, '0', 'E']
01 RE [True, '01', 'RE'] [True, '0', 'E']
01 RR [True, '010', 'RER'] [True, '0', 'E']
10 EE [True, '1001', 'RERE'] [True, '10', 'EE']
10 ER [True, '100', 'RER'] [True, '1', 'E']
10 RE [True, '10', 'RE'] [True, '1', 'E']
10 RR [True, '101', 'RER'] [True, '1', 'E']
11 EE [True, '1010', 'RERE'] [True, '11', 'EE']
11 ER [True, '101', 'RER'] [True, '1', 'E']
11 RE [True, '110', 'RRE'] [False]
11 RR [True, '11', 'RR'] [False]


In [16]:
testGPW_S_on_GPW(deltas, thetas, steps, "", True)

je tam R^iE a^i
je tam R^iE a^i
je tam R^iE a^i
delta = 00, theta = RR : [True, '0', 'R'] 

je tam R^iE a^i
je tam R^iE a^i
delta = 01, theta = RE : [True, '1', 'R'] 
je tam RR aa*

je tam R^iE a^i
je tam R^iE a^i
delta = 10, theta = RE : [True, '1', 'R'] 
je tam RR aa*

je tam R^iE a^i
je tam R^iE a^i
je tam R^iE a^i
delta = 11, theta = RR : [True, '0', 'R'] 

testGPW_S_on_GPW function took 1.239 ms


In [17]:
"ahoj"[:0]

''

In [18]:
a = 10
b = 3
(a<b) or (b!=3)

False